In [ ]:

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import kagglehub
from sklearn.model_selection import train_test_split
import requests
import os
import concurrent.futures

Creating the directories that will receive the images

In [1]:
def check_directories():
    os.makedirs('images/dogs', exist_ok=True)
    os.makedirs('images/ducks', exist_ok=True)
    os.makedirs('images/foxes', exist_ok=True)

check_directories()  

NameError: name 'os' is not defined

In [ ]:
def get_images(api_url):
    r = requests.get(api_url)
    data = r.json()

    if(api_url == 'https://randomfox.ca/floof/'):
        second_key = list(data.keys())[0]
    else:
        second_key = list(data.keys())[1]
    
    return data[second_key]



In [ ]:

def download_images(i,animal):

    try:
        while True:
            if(animal == 'dog'):
                url = get_images('https://random.dog/woof.json')
            elif(animal == 'foxe'):
                url = get_images('https://randomfox.ca/floof/')
            elif(animal == 'duck'):
                url = get_images('https://random-d.uk/api/random?type=jpg')
            
         
            img_data = requests.get(url)
           
            if url.endswith(('.mp4', '.webm', 'gif')):
                print(f"[{i}] Video/Gif detected , trying again...")
                continue 

            file_path = f'images/{animal}s/{animal}_{i}.jpg'

            with open(file_path, 'wb') as f:
                f.write(img_data.content)

            print(f"Successfully downloaded: {file_path}")
            break
    except Exception as e:
        print(f"error: {e}")
        raise


total_images = 1000

check_directories()

animals = ['foxe','duck','dog']



with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
    for animal in animals:
        futures = [executor.submit(download_images, i, animal) for i in range(1, total_images + 1)]
        concurrent.futures.wait(futures)



In [ ]:
from PIL import Image
import numpy as np


animals = ['foxe','duck','dog']

x_dict = {
    'foxe': [],
    'duck': [],
    'dog': []
}

y_dict = {
    'foxe': [],
    'duck': [],
    'dog': []
}



In [ ]:
x_dict['foxe'] = []
x_dict['duck'] = []
x_dict['dog'] = []

y_dict['foxe'] = []
y_dict['duck'] = []
y_dict['dog'] = []


In [ ]:

for animal in animals:

  for i in range(1,1001):
    

      file_path = f'images/{animal}s/{animal}_{i}.jpg'      
      image = Image.open(file_path).convert('RGB')
      image = image.resize((64,64))
      image_array = np.array(image)
      x_dict[animal].append(image_array)
      y_dict[animal].append(f'{animal}')



In [ ]:

X = x_dict['foxe'] + x_dict['duck'] + x_dict['dog']
y = y_dict['foxe'] + y_dict['duck'] + y_dict['dog']



In [ ]:
result = 0

for images in X:
    if images.shape != (64,64,3):
        images = images[:,:,3]
        result += 1



In [ ]:
from sklearn.linear_model import LogisticRegression

X = X.reshape((X.shape[0], -1))  


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

# instantiate the model
logreg = LogisticRegression(solver='liblinear', random_state=42)

# fit the model
logreg.fit(X_train, y_train)

In [ ]:
y_pred = logreg.predict(X_test)


In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy in test set: {accuracy:.2f}')

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=animals, yticklabels=animals)
plt.xlabel("Predicted")
plt.ylabel("Real")
plt.title("Confusion Matrix")
plt.show()
